# NCBI Datasets and ElasticBLAST - BOSC/CoFest (07/15/2022)

### Table of contents
* [Part I: Introduction to NCBI Datasets](#Part-I) 
* [Part II: Accessing metadata](#Part-II)
* [Part III: Accessing genomes](#Part-III)
* [Part IV: Accessing genes](#Part-IV)
* [Part V: Accessing orthologs](#Part-V)
* [Part VI: Using ElasticBLAST and Datasets](#Part-VI)

### Important resources
- Github: https://github.com/ncbi/datasets/tree/master/training/BOSC_2022
- NCBI datasets: https://www.ncbi.nlm.nih.gov/datasets/
- ElasticBLAST: https://blast.ncbi.nlm.nih.gov/doc/elastic-blast/
- jq cheat sheet: https://github.com/ncbi/datasets/blob/master/training/BOSC_2022/jq_cheatsheet.md

## Case study: Elmo loves ants

Elmo is a graduate student at the Via Sesamum University. As part of his Ph.D. project, he studies Panamanian leaf cutter ants (genus *Acromyrmex*, family Formicidae) and how variation in the gene *orco* (**o**dorant **r**eceptor **co**receptor) affects the colonies of this genus.

(here's the [link](https://www.ncbi.nlm.nih.gov/labs/pmc/articles/PMC5556950/) to a cool paper talking about this gene in ants of the species *Ooceraea biroi*).

Elmo will use `datasets` to help him gather the existing genomic resources from NCBI. He will:

- download all available genomes for the genus *Acromyrmex*
- download the *orco* gene from the *Acromyrmex* reference genome
- download the ortholog set for this gene for all ants (Formicidae)

In addition, he will use `BLAST` and `ElasticBLAST` to do the following tasks:
- Download the NCBI taxonomy database 
- Create a custom BLAST database with taxonomy information
- Prepare an ElasticBLAST search on the cloud
- Submit an ElasticBLAST search, download and visualize the results
- Cleanup after the search is done


## Part I: Introduction to NCBI Datasets<a class="anchor" id="Part-I"></a>

### How is `datasets` organized?

[NCBI datasets](https://www.ncbi.nlm.nih.gov/datasets/docs/v1/download-and-install/) is a command line tool that allows users to download data packages (data + metadata) or look at metadata summaries for genomes, RefSeq annotated genes, curated ortholog sets and SARS-Cov-2 virus sequences and proteins. The program follows a hierarchy that makes it easier for users to select exact which options they would like to use. In addition to the program commands, additional flags are available for filtering the results. We will go over those during this tutorial.
<img src="https://www.ncbi.nlm.nih.gov/datasets/docs/v1/datasets_schema_complete.svg" alt="datasets" style="width: 800px;"/>

In addition to `datasets`, we will be using `jq` (JSON parser) to take a look at the metadata information. Our metadata reports are almost all in JSON or [JSON Lines](https://jsonlines.org/) format. We put together a [jq cheat sheet]( https://github.com/ncbi/datasets/blob/workshop-cshl-2021/training/cshl-2021/jq_cheatsheet.md) to help you extract information from those files.  

For example: if I want to download the cat reference genome (<i>Felis catus</i>), I would use the command below:

In [ ]:
%%bash
#Download and unzip the data package for D. melanogaster (taxid 7227) reference genome, 
#with associated annotation files and metadata

datasets download genome taxon 7227 --reference --no-progressbar --filename dmel_ref.zip
unzip dmel_ref.zip -d dmel

Instead of downloading a data package, I could instead look at the metadata information by using the `summary`command. Here, I'm pipping it to [`jq`](https://stedolan.github.io/jq/) so it's easier to read:

In [ ]:
%%bash
#Check the metadata information for the Dmel reference genome

datasets summary genome taxon 7227 --reference | jq .

### Data packages

NCBI Datasets delivers data as <u>data packages</u>, which which are zip archives containing both data (FASTA, GFF3, GTF, GBFF) and metadata files (JSON, JSON-Lines). The image below shows the contents of the genome data package. Files are included depending on their availability. For example: for an annotated genome, the data package would include FASTA files (genomic, transcript, protein and CDS sequences) and annotation files (GFF3, GTF and GBFF).


<img src="https://github.com/ncbi/datasets/blob/487ea526b5f398f3958a1ce122d81f3e83340bc0/training/BOSC_2022/images/genome_data_package.png?raw=true" alt="data_package" />

### How to get help when using the command line

Since `datasets` is a very hierarchical program, we can use that characteristic to our advantage to get very specific help.   For example: if we type `datasets --help`, we will see the first level of commands available.


In [ ]:
%%bash
datasets --help

Notice the difference from when we type `datasets summary genome taxon formicidae --help`  


In [ ]:
%%bash
datasets summary genome taxon carnivora --help

## Part II: Accessing metadata<a class="anchor" id="Part-II"></a>

For example, let's figure out what kind of genome information NCBI has for red pandas (family Ailuridae).

In [ ]:
%%bash
# Get metadata info

datasets summary genome taxon ailuridae

In [ ]:
%%bash
# Get metadata info and save to a file

datasets summary genome taxon ailuridae > ailuridae_summary.json

**Now let's take a look at the metadata using jq**

In [ ]:
%%bash

datasets summary genome taxon ailuridae | jq . 

**EXERCISE**: Now it's your turn. Choose a taxon (or genome accession) of your interest and use `datasets` to take a quick look at the metadata. Do you see anything interesting?

In [ ]:
%%bash
# Get metadata info for



### A little bit more about json files
A JSON (JavaScript Object Notation) file stores data structures and objects. In a very simplified (and non-technical) way, a JSON file is a box, that might contain other boxes with more boxes inside. In `datasets summary genome` our JSON "box" is organized like this:

<img src="https://github.com/ncbi/datasets/blob/master/training/BOSC_2022/images/json8.png?raw=true" alt="image"/>

But let's explore the "boxes" in stages, so we can understand how everything is organized and how we can use this knowledge to extract information from the summary metadata file. At the first level, we have this: 
```
{
 assemblies[
      assembly{},
      assembly{},
 ],
 total_count
}
```
<img src="https://github.com/ncbi/datasets/blob/master/training/BOSC_2022/images/json1.png?raw=true" />

If we want to look at the value in the field "total_count", here's the command we would use:

In [ ]:
%%bash
datasets summary genome taxon herpestidae | jq '.total_count'

If we continue to expand each one of those assembly boxes, more levels of the hierarchy will be revealed. Let's expand each assembly and look at what information we can find at that level.  

<img src="https://github.com/ncbi/datasets/blob/master/training/BOSC_2022/images/json2a.png?raw=true" alt="image"/>

Here we can see that some of the assembly information, such as assembly accession number, contig N50 or submission date are not included inside any of the available "boxes" (annotation_metadata, chromosomes, bioproject_lineage, and org). Those fields describe assembly features/characteristics that pertain to the entire assembly, and not only any of those boxes available. What are the contig n50 values of those assemblies?

To retrieve that information, we need to call each box, starting from the largest one, until the field we're interested in. And each level is separated from the next by a period (.). 

<img src="https://github.com/ncbi/datasets/blob/master/training/BOSC_2022/images/json3.png?raw=true" />

In [ ]:
%%bash
datasets summary genome taxon herpestidae | jq '.assemblies[].assembly.contig_n50'

Now let's see what we have inside `annotation_metadata`, `bioproject_lineages`, `org` and `chromosomes`. 
<img src="https://github.com/ncbi/datasets/blob/master/training/BOSC_2022/images/json8.png?raw=true" alt="image"/>

Now let's see how we can retrieve the scientific names associated with those assemblies.
<img src="https://github.com/ncbi/datasets/blob/master/training/BOSC_2022/images/json4.png?raw=true" />

In [ ]:
%%bash
datasets summary genome taxon herpestidae | jq '.assemblies[].assembly.org.sci_name'

As you can see, `jq` is very useful in retrieving information from the summary metadata *as long as* you know the path to find it. Let's try a few more complex examples.
<img src="https://github.com/ncbi/datasets/blob/master/training/BOSC_2022/images/json5.png?raw=true" />

First, let's retrieve information from three fields at the same time: scientific name (`sci_name`), assembly accession number (`assembly_accession`) and contig N50 (`contig_n50`).

In [ ]:
%%bash
datasets summary genome taxon herpestidae | jq '.assemblies[].assembly 
| (.org.sci_name, .assembly_accession, .contig_n50)'

Since all three fields are inside the `.assemblies[].assembly`, we can call the first part of the path once and use a pipe (|) to call each specific field. 
Now let's try to make this a little easier to read. We can create new fields and assign values to them, like this:

In [ ]:
%%bash
datasets summary genome taxon herpestidae | jq '.assemblies[].assembly 
| {species: .org.sci_name, accession: .assembly_accession, contigN50:.contig_n50}'

**Last one**: let's look at a larger collection of genome assemblies (let's say, all Carnivora) and select only those assemblies with contig N50 larger than 15 Mb (15000000 bp). `datasets` provides many options for filtering, but there is no built-in filter for contig N50 size.  

Here's what we want to see: assembly accession number, species and assembly level for those genomes with contig N50 above 15 Mb.

In [ ]:
%%bash
datasets summary genome taxon carnivora | jq -r '.assemblies[].assembly 
| select(.contig_n50 > 15000000) 
| [.assembly_accession, .org.sci_name, .assembly_level] 
| @tsv'

**RESOURCE:**  
We included a list of all fields in the genome summary in our [jq cheatsheet](https://github.com/ncbi/datasets/blob/master/training/BOSC_2022/jq_cheatsheet.md) to help you extract the information you need. And we will show you now how to do that. 

## Part III: Accessing genomes<a class="anchor" id="Part-III"></a>

Independent of choosing `datasets download` or `datasets summary`, there are two options for retrieving genome information:
- accession (assembly accession and version)
- taxon (common name, scientific name or NCBI taxon-ID at any taxonomic level)

Additional flags, such as `--assembly-source`, `--reference` or `--annotated`, will help narrow down to the genomes of interest.

`datasets download genome accession GCF_018350175.1`  
`datasets download genome taxon cat --reference`  

Both commands will download the same genome package: cat (<i>Felis catus</i>) <u>reference genome</u>. 

### Let's continue to explore the available genomes for the family Formicidae

For this part, we will use two UNIX commands: `sort` and `uniq`. 

- `sort` can be used to sort text files line by line, numerically and alphabetically.   
- `uniq` will filter out the repeated lines in a file. However, `uniq` can only detect repeated lines if they are adjacent to each other. In other words, if they are alphabetically or numerically sorted. The flag `-c` or `--count` tells the command `uniq` to remove the repeated lines, and to count how many times each value appeared. 

So, we will use `jq` to extract the information we need, sort the result and count the number of unique entries.

In [ ]:
%%bash
# For which species does NCBI have genomes in its database? How many per species?

datasets summary genome taxon formicidae | jq '.assemblies[].assembly.org.sci_name' | sort | uniq -c

In [ ]:
%%bash
# What is the assembly level (contig, scaffold, chromosome, complete) breakdown?

datasets summary genome taxon formicidae | jq '.assemblies[].assembly.assembly_level' | sort | uniq -c

Now that we explored the number of available genomes, as well as the assembly level and other important characteristics, it's time to download the genomes for Elmo's research project.

First, we want to download all genomes for the genus *Acromyrmex* from Genbank. 

In [ ]:
%%bash
# Download all available GenBank assemblies for the genus Acromyrmex and save as genomes.zip
datasets download genome taxon acromyrmex --assembly-source genbank --filename genomes.zip --no-progressbar

The next step is to unzip the data package zip file to a new folder called genomes

In [ ]:
%%bash
# Unzip genomes.zip to the folder genomes
unzip -o genomes.zip -d genomes

Finally, let's take a look at the contents of the data package with the command `tree`

In [ ]:
%%bash
# Explore the folder structure of the folder genome with the command tree
tree -C genomes/

### Let's recap our goals

We used `datasets` to download all the Genbank assemblies for the genus *Acromyrmex*. The next step is to download the gene *orco* (odorance receptor coreceptor) for the same genus. But first, let's learn more about how genes are organized at NCBI.

## Part IV: Accessing genes <a class="anchor" id="Part-IV"></a>
### GENES

Independent of choosing `datasets download` or `datasets summary`, there are three options for retrieving gene information:
- accession
- gene-id
- symbol

When choosing any of those three options, you will retrieve the gene information for the **reference** taxon. Like this:

`datasets download gene accession XR_002738142.1`  
`datasets download gene gene-id 101081937`  
`datasets download gene symbol BRCA1 --taxon cat`  

All three commands will download the same gene from the cat (<i>Felis catus</i>) <u>reference genome</u>. 

#### accession
Unique identifier. Accession includes RefSeq RNA and protein accessions. Since it's unique, taxon is implied (aka there will never be two sequences from different taxa with the same accession number).  

#### gene-id
Also a unique identifier. Every RefSeq genome annotated has a unique set of identifiers. For example: the gene-id for BRCA1 in human is 672, while in cat is 101081937.  

#### symbol
Differently from accession and gene-id, gene symbol is not unique and means different things in different taxonomic groups. If using the symbol option, you should specify the species. The default option is human.

**Remember**: both `summary` and `download` will return results for the **reference assembly** of a <u>single species</u>. If you want to download a curated set of the same gene for multiple taxa, you should use the option `ortholog`. We'll talk more about it later. For reference, here's the JSON organization of the gene summary metadata.  

<img src="https://github.com/ncbi/datasets/blob/master/training/BOSC_2022/images/gene_json.drawio.png?raw=true" />

Now let's take a look at a gene example:

In [ ]:
%%bash
#Example: IFNG in human
datasets summary gene symbol ifng | jq -C .


In [ ]:
%%bash
# how datasets deals with synonyms
datasets summary gene symbol IFG | jq -C -r '.genes[].gene | {species: .taxname, symbol: .symbol, synonyms:.synonyms}'


In [ ]:
%%bash
#Example: IFNG in cat
datasets summary gene symbol ifng --taxon "felis catus"


### Back to ants
We will download the gene *orco* for the species *Acromyrmex echinatior*. We will use the gene-id 105147775 instead of the symbol because no informative gene symbol has been assigned for this gene.  

In [ ]:
%%bash

# Using gene-id to retrieve gene information
datasets summary gene gene-id 105147775 | jq -C '.genes[].gene 
| {gene_description: .description, gene_id: .gene_id, symbol: .symbol, species: .taxname}'

In [ ]:
%%bash

# if we try to retrieve metadata information for this gene using the symbol orco, what happens?
datasets summary gene symbol orco --taxon "acromyrmex echinatior"


In [ ]:
%%bash

# Download the gene data package for the gene-id 105147775 (*orco* in Acromyrmex echinatior)
datasets download gene gene-id 105147775 --filename gene.zip --no-progressbar


In [ ]:
%%bash
#Unzip the file
unzip -o gene.zip -d gene

In [ ]:
%%bash
#Explore the data package structure using tree
tree gene

Now we are going to take advantage of the fact that we are using a Jupyter Notebook and use the package `pandas` to look at the gene data table

In [ ]:
import pandas as pd                                                        #load pandas to this notebook
gene_orco = pd.read_csv('gene/ncbi_dataset/data/data_table.tsv', sep='\t') #use pandas to import the data_table.tsv
gene_orco                                                                  #visualize the data table as the object gene_orco

## Part V: Accessing orthologs <a class="anchor" id="Part-V"></a>

### Orthologs

The options to retrieve ortholog sets are the same as those for genes. We'll go over the differences when using each option:

- accession
- gene-id
- symbol

When choosing any of those three options, you will download the **full ortholog set** to which they belong (unless you use additional filtering. We'll cover it below). Like this:

`datasets download ortholog accession XR_002738142.1`  
`datasets download ortholog gene-id 101081937`  
`datasets download ortholog symbol BRCA1 --taxon cat`  

All three commands will download the **same** ortholog set. 

---

#### <font color='blue'>Wait, but what is an ortholog set?</font>

>An ortholog set, or ortholog gene group, is a group of sequences that have been identified by the NCBI genome annotation team as homologous genes related to each other by speciation events. They are identified by a combination of protein similarity + local syntheny information. 
Currently, NCBI has ortholog sets calculated for vertebrates and some insects. 


#### accession
Unique identifier. Accession includes RefSeq accession RNA and protein sequences. Since it's unique, taxon is implied (aka there will never be two sequences from different taxa with the same accession number).

#### gene-id
Also an unique identifier. Every RefSeq genome annotated has a unique set of identifiers. For example: the gene-id for BRCA1 in human is 672, while in cat is 101081937. You can use either one (672 or 101081937) to get the same vertebrate BRCA1 ortholog set.

#### symbol
Differently from accession and gene-id, gene symbol is not unique and means different things in different taxonomic groups. For example: the P53 ortholog set in vertebrates is different from the insect set. If using the symbol option, you should specify the taxonomic group. The default option is human. Note that if you want ortholog sets from multiple vertebrate species, you might end up downloading the same ortholog set multiple times. Like this: 

`datasets download ortholog symbol brca1 --taxon cat`  
`datasets download ortholog symbol brca1 --taxon chicken`  
`datasets download ortholog symbol brca1 --taxon "chelonia mydas"`  

If that's the case, how do you filter the ortholog set to include *only* your taxonomic group of interest?

### Applying a taxonomic filter to the ortholog set

For the orthologs, `datasets` provides the flag `--taxon-filter`, which allows the user to restrict the summary or download to one or multiple taxonomic groups.  `--taxon` and `--taxon-filter` have different effects on the data package/summary output. A few examples:

- `datasets summary ortholog symbol brca1 --taxon-filter "felis catus"`  
Prints a json metadata summary of the gene brca1 for the domestic cat. 
We did not specify a `--taxon` because the default is human, and Felidae and human are part of the same brca1 ortholog set.   

  

- `datasets summary ortholog symbol brca1 --taxon "felis catus"`  
Even though this option looks almost the same as the one above, the result is *very different*. Here, we're asking `datasets` to find the ortholog set to which the gene brca1 in the domestic cat belongs. And `datasets` will download the <u>entire</u> ortholog set, not only the sequences for the domestic cat.


- `datasets summary ortholog symbol brca1 --taxon "felis catus" --taxon-filter "felis catus"`  
gives you the same result as `datasets summary ortholog symbol brca1 --taxon-filter "felis catus"`


The summary metadata for orthologs is presented in JSON Lines, which means that each gene entry is in a different line. Here's the diagram to help you create queries.
  
<img src="https://github.com/ncbi/datasets/blob/master/training/BOSC_2022/images/ortholog_jsonl.drawio.png?raw=true" />

#### We are going to do the following steps:
- download the ortholog data package and save it with the name ortholog.zip
- unzip it to the folder ortholog
- look at the files

Helpful info:

- gene symbol: orco
- gene-id in *Drosophila melanogaster*: 40650
- gene-id in *Acromyrmer echinatior*: 105147775
- target taxon: Formicidae

In [ ]:
%%bash
# download the orco ortholog set for ants (Formicidae)
datasets download ortholog gene-id 40650 --taxon-filter formicidae --filename ortholog.zip --no-progressbar


In [ ]:
%%bash
# unzip it to the folder ortholog
unzip -o ortholog.zip -d ortholog


In [ ]:
%%bash
#Explore the folder structure
tree -C ortholog/


In [ ]:
# Create an object called ortho_table using pandas
ortho_table = pd.read_csv("ortholog/ncbi_dataset/data/data_table.tsv", sep='\t')
ortho_table

## What have we done so far?
- Explored metadata for all ant genomes
- Downloaded genomes for the panamanian leaf cutter ant
- Downloaded the orco gene for Acromyrmex echinatior
- Downloaded the ortholog set for all ants for the orco gene


# Part VI: ElasticBLAST and Datasets<a class="anchor" id="Part-VI"></a>

In [ ]:
from uuid import uuid4
import os

## Download sequences

If you have already done [Part III: Accessing genomes](#Part-III) and [Part IV: Accessing genes](#Part-IV), you can skip to [Create a new folder for the blast search](#new-folder). If not, follow the steps below.


In [ ]:
%%bash
# Download the gene data package for the gene-id 105147775 (*orco* in Acromyrmex echinatior)
datasets download gene gene-id 105147775 --filename gene.zip --no-progressbar
unzip -o gene.zip -d gene

In [ ]:
%%bash
# Download all available GenBank assemblies for the genus Acromyrmex and save as genomes.zip
datasets download genome taxon acromyrmex --assembly-source genbank --filename genomes.zip --no-progressbar
unzip -o genomes.zip -d genomes

### Create a new folder for the blast search <a class="anchor" id="new-folder"></a>

In [ ]:
%%bash

mkdir blast

### Download NCBI taxonomy database <a class="anchor" id="taxonomy"></a>

In [ ]:
# change directory to the folder blast
%cd blast

# download the NCBI Taxonomy Database (taxdb)
!update_blastdb taxdb

# back to the home directory
%cd ~

## Create a BLAST database with taxonomy information

### Concatenate downloaded genomic sequences in one file

In [ ]:
%%bash
# Concatenate all genomic sequence files into one
cat genomes/ncbi_dataset/data/GCA_000204515.1/unplaced.scaf.fna \
  genomes/ncbi_dataset/data/GCA_017607455.1/GCA_017607455.1_ASM1760745v1_genomic.fna \
  genomes/ncbi_dataset/data/GCA_017607545.1/GCA_017607545.1_ASM1760754v1_genomic.fna \
  genomes/ncbi_dataset/data/GCA_017607565.1/GCA_017607565.1_ASM1760756v1_genomic.fna > blast/genomes.fa

### Create taxonomy map for the genomic sequences

In [ ]:
%%bash
cat genomes/ncbi_dataset/data/GCA_000204515.1/unplaced.scaf.fna | awk '/>/ {print substr($1, 2, length($1)-1) "\t103372";}' > blast/taxids.tsv
cat genomes/ncbi_dataset/data/GCA_017607455.1/GCA_017607455.1_ASM1760745v1_genomic.fna | awk '/>/ {print substr($1, 2, length($1)-1) "\t230686";}' >>blast/taxids.tsv
cat genomes/ncbi_dataset/data/GCA_017607545.1/GCA_017607545.1_ASM1760754v1_genomic.fna | awk '/>/ {print substr($1, 2, length($1)-1) "\t2715315";}' >>blast/taxids.tsv
cat genomes/ncbi_dataset/data/GCA_017607565.1/GCA_017607565.1_ASM1760756v1_genomic.fna | awk '/>/ {print substr($1, 2, length($1)-1) "\t230685";}' >>blast/taxids.tsv
head blast/taxids.tsv

### Create a BLAST database

In [ ]:
# Change to blast directory
%cd blast

#Create a blast database
!makeblastdb -in genomes.fa -out genomesdb -dbtype nucl -parse_seqids -taxid_map taxids.tsv

#Change back to the home directory
%cd ~ 

## Prepare ElasticBLAST search

### ElasticBLAST results bucket

ElasticBLAST uses a cloud storage bucket to store a BLAST database and search results. If you already have an S3 bucket assign its URI to `RESULTS` variable. Otherwise, running this cell without modifications will generate a name with a random suffix.

Remember that S3 bucket names must be globally unique. You cannot use a bucket name if one already exists.

In [ ]:
YOURNAME = f'{str(uuid4())[:8]}'
RESULTS = f's3://elasticblast-{YOURNAME}'
print(f'Your results bucket: {RESULTS}')
!echo {RESULTS} >results_bucket

### Create results bucket

Skip this step if the bucket already exists.

In [ ]:
!aws s3 mb {RESULTS}

### Upload your BLAST database

ElasticBLAST searches BLAST databases stored in cloud storage. Upload your database to your cloud bucket.

In [ ]:
!aws s3 cp blast {RESULTS}/db --recursive --exclude "*" --include "genomesdb.*"

### ElasticBLAST configuration

ElasticBLAST uses a configuration file to specify search parameters, such as query, database, search program, and parameters. For easier code management we will assign ElasticBLAST configuration file name to the variable `conf_file`.

In [ ]:
conf_file = 'elastic-blast-config.ini'

The cell below will write the contents of ElasticBLAST configuration file.

The configuration file instructs ElasticBLAST to do a BLASTN search of query sequences in file `gene/ncbi_dataset/data/gene.fna` against the database in your results bucket.

In [ ]:
# These are the contents of ElasticBLAST config 
conf = f"""
[cluster]
num-nodes = 1

[blast]
program = blastn
db = {RESULTS}/db/genomesdb
queries = gene/ncbi_dataset/data/gene.fna
options = -evalue 1e-50 -outfmt 11 -max_hsps 1
"""

# Write the config to the file: elastic-blast-config.ini
with open(conf_file, 'w') as f:
    f.write(conf)

## Submit an ElasticBLAST search

We can now submit the ElasticBLAST search, run the cell below. The submission may take a few minutes.

In [ ]:
os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'
os.environ['ELB_DISABLE_AUTO_SHUTDOWN'] = '1'
!elastic-blast submit --cfg {conf_file} --results {RESULTS}

## Check search status

The cell below checks search status. ElasticBLAST splits query sequences into batches. The elastic-blast status command shows how many of these batches are pending, running, completed, or completed. When the whole search is done you will see only the message: "Your ElasticBLAST search succeeded ..." or "Your ElasticBLAST search failed ..."

In [ ]:
!elastic-blast status --results {RESULTS}

## Wait until the search is done

Run the cell below to wait until the search is done. The cell will keep working until ElasticBLAST search is complete.

In [ ]:
!elastic-blast status --results {RESULTS} --wait

We will change working directory to blast.

In [ ]:
%cd blast

## Download and uncompress results

When an ElasticBLAST search is done, search results are placed in your results bucket. Now we need to download and uncompress the results to analyze them.

In [ ]:
!aws s3 cp {RESULTS} . --exclude "*" --include "*.out.gz" --recursive
!gzip -d -f batch_*.gz

### Convert results to the tabular format

In [ ]:
%%bash
blast_formatter \
-archive batch_000-blastn-genomesdb.out \
-outfmt '6 sseqid sstart send evalue length staxid ssciname' > orco_acromyrmex_1e-50.tsv
head orco_acromyrmex_1e-50.tsv

#### Converting from BLAST to fasta

Now we are going to use some "tricks" (not really, just some good old bash scripting) to extract fasta sequences from the BLAST output. For tthis task, we will be using `blast_formatter` again.

In [ ]:
%%bash
# Convert BLAST output to fasta

blast_formatter \
-archive batch_000-blastn-genomesdb.out \
-outfmt '6 ssciname sseqid sseq' \
-max_target_seqs 4 | awk 'BEGIN{FS="\t"; OFS="\n"}{gsub(/ /, "_", $1);gsub(/-/, "", $3); print ">"$1"_"$2,$3}' > ../acromyrmex_orco.fasta


### VERY IMPORTANT!
For the next steps, we need to go back to our home folder. Let's do it in steps again.

In [ ]:
%%bash
## Check where you are
pwd

In [ ]:
## If you're not in the home folder, run this command:
%cd ~

### Multiple sequence alignment: BLAST matches + *orco* orthologs

First, let's simplify the FASTA headers in the ortholog set. Since we will be building a phylogeny, we want the names to be easy to read.

First, we're extract the FASTA headers and save it as a text file.

In [ ]:
%%bash

grep ">" ortholog/ncbi_dataset/data/gene.fna > ortholog_seqid.txt
head ortholog_seqid.txt

Now let's create a file with two columns: column 1 will be the original name and column 2 will be what we want the FASTA headers to be. 

In [ ]:
%%bash

rm name_map.tsv
cat ortholog_seqid.txt | while read line; do
    accession=$( echo "$line" | grep -o -P '^>\K[^:$]+') #extract accession
    species=$( echo "$line" | grep -o -P '(?<=organism\=).*?(?=])' | sed 's/ /\_/g' ) #extract species name and replace space by underscore
    geneid=$(echo "$line" | grep -o -P '(?<=GeneID\=).*?(?=])' )  #extract the GeneID
    old=$( echo $line | sed 's/ /\_/g;s/>//g')  #replace space by underscore and remove the ">"
    printf "${old}\t${species}-${accession}-${geneid}\n" >> name_map.tsv ; 
done


Now let's make a copy of the ortholog sequence file (so we can keep the original) and change the FASTA header by replacing spaces by underscores.

In [ ]:
%%bash

#Copy the ortholog dataset fasta
cp ortholog/ncbi_dataset/data/gene.fna ortholog_gene.fna

#Remove spaces in the fasta sequnce names
sed 's/ /_/g' ortholog_gene.fna > ortholog_gene_nospaces.fna


Finally, let's use [`seqkit`](https://bioinf.shenwei.me/seqkit/usage/) to replace the FASTA headers.

In [ ]:
%%bash

#Replace the names in the fasta file
cat ortholog_gene_nospaces.fna | seqkit replace \
--kv-file  <(cut -f 1,2 name_map.tsv) \
--pattern "^(.*)" --replacement "{kv}" > ortholog_gene_final.fna

Let's check the final FASTA headers

In [ ]:
%%bash

grep ">" ortholog_gene_final.fna

### Multiple sequence alignment and phylogenetic reconstruction

Now, let's concatenate the FASTA we extracted from the BLAST matches, align them using MAFFT and use FastTree to generate an approximate ML phylogeny.

In [ ]:
%%bash

#Concatenate sequences. 
cat ortholog_gene_final.fna acromyrmex_orco.fasta > orco_all.fasta

#align sequences with mafft
mafft orco_all.fasta > orco_all_aln.fasta

#Generate a phylogeny using fasttree
FastTree -nt orco_all_aln.fasta > orco.tree

### Visualizing the tree

In [ ]:
# We will use the package toytree to look at the phylogenetic tree we just created

import toytree
orco_tree = toytree.tree("orco.tree")
orco_tree_rooted = orco_tree.root(names=["Odontomachus_brunneus-NW_022639457.1-116854080","Dinoponera_quadriceps-NW_014554446.1-106748868","Harpegnathos_saltator-NW_020230404.1-105183395"])
orco_tree_rooted.draw(tree_style='d')

## Cleanup

### Delete cloud resources created by ElasticBLAST

If you did not enable ElasticBLAST auto-shutdown feature, cloud resources (like AWS Batch queue and compute environment) have to be deleted manually via the command below:

In [ ]:
!elastic-blast delete --results {RESULTS}

### Delete cloud bucket

If you do not need BLAST search results stored in the cloud, delete the cloud bucket so that you are not charged for it.

In [ ]:
!aws s3 rb {RESULTS} --force